<img src="../.images/logosnb.png" alt="Banner" style="width: 1100px;"/>

<div style='color: #690027;' markdown="1">
    <h1>CLASSIFICATIE VAN DE IRIS DATASET MET EEN HIDDEN LAYER </h1> 
</div>

<div class="alert alert-box alert-success">
De classificatie in drie klassen gebeurt op basis van twee kenmerken.
</div>

Deze notebook bevat een concreet voorbeeld van een neuraal netwerk. Het netwerk wordt opgebouwd met de functionaliteiten van de Python-module Keras. 

Het voorbeeld betreft een classificatieprobleem, nl. het classificeren van drie soorten irissen. Van elke iris worden 4 kenmerken gegeven: de lengte en breedte van een kelkblad en de lengte en breedte van een kroonblad, opgemeten in centimeter. Er zijn 3 categorieën: <em>iris setosa</em>, <em>iris versicolor</em> en <em>iris virginica</em>. 

Om het netwerk te trainen, wordt gebruik gemaakt van de Iris dataset [1][2]. Deze dataset bestaat uit 150 monsters.  

<table><tr>
<td><img src="../.images/IntroductieMachineLearning/Kosaciec_szczecinkowaty_Iris_setosa.jpg" alt="Drawing" style="width: 200px;"/></td>
<td><img src="../.images/IntroductieMachineLearning/Iris_versicolor_3.jpg" alt="Drawing" style="width: 220px;"/></td>
<td><img src="../.images/IntroductieMachineLearning/Iris_virginica.jpg" alt="Drawing" style="width: 203px;"/></td>
</tr></table>

<table><tr>
    <td><em>Iris setosa</em> [3]</td>
<td> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</td>
    <td><em>Iris versicolor</em> [4]</td>
<td>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</td>
    <td><em>Iris virginica</em> [5]</td>
</tr></table>
<br>
<center>Figuur 1: <em>Iris setosa</em> door Radomil Binek [CC BY-SA 3.0 (https://creativecommons.org/licenses/by-sa/3.0)], via Wikimedia Commons;<br> <em>Iris versicolor</em>. No machine-readable author provided. Dlanglois assumed (based on copyright claims). CC BY-SA 3.0, via Wikimedia Commons. <br> <em>Iris virginica</em>  door Frank Mayfield [CC BY-SA 2.0 (https://creativecommons.org/licenses/by-sa/2.0)], via Wikimedia Commons.</center>

### De nodige modules importeren

In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler        # om data te standaardiseren
from sklearn.utils import shuffle                       # om data onderling te kunnen wisselen

# import keras
from keras import models
from keras import layers
from keras.utils import to_categorical

<div style='color: #690027;' markdown="1">
    <h2>1. Inlezen van de data</h2> 
</div>

In [ ]:
iris = pd.read_csv("../.data/IntroductieDeepLearning/iris.dat", header=None)  # in te lezen tabel heeft geen hoofding

<div style='color: #690027;' markdown="1">
    <h2>2. Tonen van de ingelezen data</h2> 
</div>

In [ ]:
iris

Deze tabel komt overeen met een matrix met 150 rijen en 5 kolommen: <br>
150 monsters, 4 kenmerken (x1, x2, x3, x4) en 1 label (y) <br><br>
De kenmerken:<br>
- eerste kolom: lengte kelkblad 
- tweede kolom: breedte kelkblad
- derde kolom: lengte kroonblad 
- vierde kolom: breedte kroonblad<br><br>

Het label:<br>
- laatste kolom: de naam van de soort 

In [ ]:
# omzetten naar NumPy array
iris = np.array(iris)

Je zal een classificatie van de drie soorten irissen doen op basis van de lengte en de breedte van het kelkblad.

In [ ]:
X = iris[:, 0:2]    # kenmerken in eerste en tweede kolom
y= iris[:, 4]         

In [ ]:
print(X,y)

In [ ]:
# data voorbereiden
# data in X standaardiseren
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [ ]:
print(X)

In [ ]:
print(np.min(X), np.max(X))

In [ ]:
# labels numeriek weergeven
y[y == 'Iris-setosa'] = 0
y[y == 'Iris-versicolor'] = 1
y[y == 'Iris-virginica'] = 2

In [ ]:
print(y)

In [ ]:
# data shufflen
X, y = shuffle(X, y)

In [ ]:
print(X, y)

In [ ]:
# data opsplitsen in traindata, validatiedata en testdata
X_train = X[0:100, :]      # 100 trainsamples
X_val = X[100:120, :]      # 20 validatiesamples
X_test = X[120:, :]        # 30 testsamples
y_train = y[0:100]
y_val= y[100:120]
y_test= y[120:] 

In [ ]:
X_train.shape                      # 100 samples

In [ ]:
# data visualiseren

font = {'family': 'serif',
        'color':  'black',
        'weight': 'normal',
        'size': 14,
        }
color = ["green", "blue", "yellow"]
soort = ["setosa", "versicolor", "virginica"]

plt.figure(figsize=(8, 6))

for target in range(3):
    X_plot = X[y == target]
    plt.scatter(X_plot[:, 0], X_plot[:, 1], marker='o', color=color[target], label=soort[target])
plt.title("Iris dataset", fontdict=font)
plt.xlabel("lengte kelkblad", fontdict=font)
plt.ylabel("breedte kelkblad", fontdict=font)
plt.legend(loc="lower right")

plt.show()

<div style='color: #690027;' markdown="1">
    <h2>3. Architectuur van het netwerk</h2> 
</div>

In [ ]:
# architectuur netwerk     https://keras.io/getting-started/sequential-model-guide/

# 'Sequential model' is model dat bestaat uit aaneengeschakelde lagen
# netwerk met dense layers, 'dense layers'  betekent 'fully connected'
# (feed forward netwerk met) 1 hidden layer

# model moet weten welke vorm van input het kan verwachten, m.a.w. wat dimensie is van de inputpunten
# daarom wordt dit aan eerste laag van Sequential model meegegeven
# enkel aan de eerste want volgende lagen krijgen dat automatisch door voltrekken wiskundige bewerkingen 
# loss-functie moet geminimaliseerd worden m.b.v. optimizer
# met metrics ga je performantie van model na

network = models.Sequential()

network.add(layers.Dense(10, input_dim=2, activation="relu"))  # hidden layer  10 neuronen, activatie ReLU
network.add(layers.Dense(3, activation="softmax"))             # output layer  3 outputneuronen, activatie softmax
network.compile(optimizer="sgd",
                loss="categorical_crossentropy",
                metrics=["accuracy"])                          # optimizer, loss en metrics kiezen

In [ ]:
# one hot encoding
y_train = to_categorical(y_train)
y_val = to_categorical(y_val)
y_test = to_categorical(y_test)

In [ ]:
print(y_train[0])

<div style='color: #690027;' markdown="1">
    <h2>4. Trainen van het netwerk</h2> 
</div>

In [ ]:
# trainen met de methode fit() van network, m.a.w. de kenmerken en de labels op elkaar afstemmen
# 50 epochs
# steeds 10 datapunten per keer gebruiken om de kost te minimaliseren
# merk op dat na elke epoch de loss en accuracy vermeld worden
history = network.fit(X_train, y_train, epochs=50, batch_size=10, validation_data=(X_val,y_val))

<div style='color: #690027;' markdown="1">
    <h2>5. Prestatie van het netwerk</h2> 
</div>

In [ ]:
loss = history.history["loss"]
epochs = range (1 , len(loss) +1)
acc = history.history["accuracy"]
val_acc = history.history["val_accuracy"]
val_loss = history.history["val_loss"]

In [ ]:
font = {'family': 'serif',
        'color':  'black',
        'weight': 'normal',
        'size': 14,
        }
plt.figure(figsize=(16,8))

plt.subplot(1,2,1)
plt.plot(epochs, loss, color="blue", label="train", linewidth=2)
plt.plot(epochs, val_loss, color="lightblue", label="val", linewidth=2)
plt.xticks(np.arange(0, 51, step=5))             
plt.title("Loss op training- en validatieset", fontdict=font)
plt.xlabel("epoch", fontdict=font)
plt.ylabel("loss", fontdict=font)
plt.legend(loc="lower left")

plt.subplot(1,2,2)
plt.plot(epochs, acc, color="green", label="train", linewidth=2)
plt.plot(epochs, val_acc, color="lime", label="val", linewidth=2)
plt.xticks(np.arange(0, 51, step=5)) 
plt.title("Accuracy op training- en validatieset", fontdict=font)
plt.xlabel("epoch", fontdict=font)
plt.ylabel("acc", fontdict=font)
plt.legend(loc="lower right")

plt.show()

In [ ]:
test_loss, test_acc = network.evaluate(X_test, y_test)
print('test_acc:', test_acc)

In [ ]:
punt = np.array([[-0.5, 3]])
voorspelling = network.predict(punt)
print(voorspelling)
voorspelling.shape

<div style='color: #690027;' markdown="1">
    <h2>6. Decision boundary</h2> 
</div>

Eerst wat voorbereidend werk. <br>
Je wilt dat uit de 'voorspelling' volgt dat de 'voorspelde klasse' gegeven wordt, voor het voorbeeld de klasse met cijfer 2.

In [ ]:
voorspelling = voorspelling[0]
print(voorspelling)

In [ ]:
max = np.max(voorspelling)
print(max)
voorspeldeklasse = np.where(voorspelling == max)
print(voorspeldeklasse)

In [ ]:
voorspeldeklasse = voorspeldeklasse[0]      
print(voorspeldeklasse)

In [ ]:
# decision boundary
font = {'family': 'serif',
        'color':  'black',
        'weight': 'normal',
        'size': 14,
        }
color = ["green", "blue", "yellow"]
soort = ["setosa", "versicolor", "virginica"]
                  
plt.figure(figsize=(8, 6))

for target in range(3):
    X_plot = X[y == target]
    plt.scatter(X_plot[:, 0], X_plot[:, 1], marker='o', color=color[target], label=soort[target])
plt.title("Classificatie Iris dataset", fontdict=font)
plt.xlabel("lengte kelkblad", fontdict=font)
plt.ylabel("breedte kelkblad", fontdict=font)

plt.legend(loc="lower right")

as1 = np.linspace(-2, 3, 40)
as2 = np.linspace(-3, 4, 40)

# resolutie
xx1 = np.arange(as1.min()-1, as1.max()+1, 0.2)
xx2 = np.arange(as2.min()-1, as2.max()+1, 0.2)                     

for a in xx1:
    for b in xx2:
        P = np.array([[a, b]])
        voorspelling = network.predict(P)[0]
        max = np.max(voorspelling)
        voorspeldeklasse = np.where(voorspelling == max)[0]
       
        if voorspeldeklasse == 0:
            kleur = "lightgreen"
        else:
            if voorspeldeklasse == 1:
                kleur = "lightblue"
            else: 
                kleur = "pink"
        plt.plot(a, b, marker='.', color=kleur)

plt.show()

<div>
    <h2>Referentielijst</h2> 
</div>

[1] Dua, D., & Karra Taniskidou, E. (2017). UCI Machine Learning Repository [http://archive.ics.uci.edu/ml]. <br> &nbsp; &nbsp; &nbsp; &nbsp; Irvine, CA: University of California, School of Information and Computer Science.<br>
[2] Fisher, R. A. (1936). The use of multiple measurements in taxonomic problems. *Annals of Eugenics*. 7(2), 179–188. <br> &nbsp; &nbsp; &nbsp; &nbsp; https://doi.org/10.1111/j.1469-1809.1936.tb02137.x.<br>
[3] Radomil Binek. [CC BY-SA 3.0 (https://creativecommons.org/licenses/by-sa/3.0)], via Wikimedia Commons.<br>
[4] Danielle Langlois. No machine-readable author provided. Dlanglois assumed (based on copyright claims).<br> &nbsp; &nbsp; &nbsp; &nbsp;  [CC BY-SA 3.0 (http://creativecommons.org/licenses/by-sa/3.0/)],
via Wikimedia Commons.<br>
[5] Frank Mayfield. [CC BY-SA 2.0 (https://creativecommons.org/licenses/by-sa/2.0)], via Wikimedia Commons.

<img src="../.images/cclic.png" alt="Banner" align="left" style="width:100px;"/><br><br>
Notebook KIKS, zie <a href="http://www.aiopschool.be">AI op School</a>, van F. wyffels & N. Gesquière is in licentie gegeven volgens een <a href="http://creativecommons.org/licenses/by-nc-sa/4.0/">Creative Commons Naamsvermelding-NietCommercieel-GelijkDelen 4.0 Internationaal-licentie</a>. 